# Simulate STREAMS for Astrophysical source  
This notebook simulates an astrophysical source with a given XSPEC spectrum in xifusim producing data streams (not Triggered) for the a given number of brightest pixels 

Note:    

1 mCrab = 90 counts/s in the 2-10 keV band and 2.0533E-11 erg/cm^2/s    
1  Crab = 90000 counts/s in the 2-10 keV band and 2.0533E-8 erg/cm^2/s  (powerlaw norm=9.5)

Possible Source spectral models:   
`Crab`  (EC)   
 - XSPEC model: TBabs*powerlaw    
 - Power law slope: $\Gamma=2.1$      
 - Foregroung absorption: $N_H=4\times 10^{21} \rm{cm^{-2}}$    
 - Power law Normalization: 9.5
 - 2-10 Flux = 2.05E-8 erg/cm2/s (1Crab)

`CasA` (EC)
 - XSEPEC model:  TBabs(powerlaw + vnei + vnei)
        

Simulation steps   
1. Read simulation parameters and derived parameters   
2. HEASOFT `xspec`: create xspec model file    
3. SIXTE `simputfile`: Create simput file with photons distribution    
4. SIXTE `sixtesim`: Run simulation to get   
    4.1 ImpactList - piximpact file for ALL photons    
    4.2 EventList - which photons (PH_ID) impact in each pixel of the detector (possibly including background and XTalk)   
    4.3 PixImpactList: piximpact file for each pixel with impacts (PH_ID) (needed by xifusim)   
5. Get list of pixels w/ impacts.   
    5.1. (Optional) For Crab extended, compare count rate of brightest pixels w/ EC's input list (+5 ct/s/pix to accomodate MXS): adjust flux_mcrab if needed   
    5.2. `xifusim`: Do single-pixel (NOMUX) xifusim simulation     
    
    

## Import routines and read parameters

In [ ]:
import os
import sys
from subprocess import run
import tempfile
import glob
from astropy.io import fits
from astropy.table import Table
import numpy as np
from xspec import Xset, Model, AllModels
import auxpileup as aux

In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = f"{tmpDir}:{os.environ['PFILES']}"
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]
verbose = 0

### Read simulation parameters   

In [ ]:
def get_parameters():
    """
    Get parameters for pairs detection analysis.
    If running in a Jupyter Notebook, use default parameters.
    If running as a script (e.g., SLURM), parse command line arguments.
    """
    
    if aux.is_notebook():
        # Default parameters for interactive use
        print("Running in notebook mode for source simulation")
        return {
            # set srcmodel parameter or xcmfile parameter
            "srcmodel": "crab", # crab or casa
            "xcmfile": "crab.xcm", # if srcmodel is not used, provide an xcm file
            "flux210_keV": None,  # if flux is not provided it will be derived from xcmfile; 1Crab flux in erg/cm2/s=2.0533E-8; 
            "exposure": 1.1, #s
            "nonxbgd": "no", #yes/no
            "XTalk": "none", # all, elec, therm, tdm_prop, tdm_prop1, tdm_prop2, tdm_der, none
            "xifusim_xml_version": "v5_20250621",
            "focus": "defoc", # defoc or infoc
            "filter": "be_filter", # be_filter, nofilt
            "verbose": 1,
            "nbrightest_pixels": 384, # number of brightest pixels to consider for defocussed Crab simulations
            "addMXS": True, # whether to add MXS contribution 
            "addJitter": True, # whether to add pointing jitter
            "decimation_factor": 10, # decimation factor for SIXTE simulation
        }
    else:
        import argparse
        parser = argparse.ArgumentParser(description="Simulate Crab spectrum with SIXTE & XIFUSIM")
        parser.add_argument("--srcmodel", type=str, choices=["crab", "casa"], default="crab", help="Spectral model to use (crab or casa)")
        parser.add_argument("--xcmfile", type=str, default=None, help="XCM file defining the spectral model")
        parser.add_argument("--flux210_keV", type=float, default=None, help="Flux in the 2-10 keV band (erg/cm2/s)")
        parser.add_argument("--exposure", type=float, required=True, help="Exposure time in seconds")
        parser.add_argument("--nonxbgd", type=str, choices=["yes", "no"], default="no", help="Include non-X-ray background (yes/no)")
        parser.add_argument("--XTalk", type=str, choices=["all", "elec", "therm", "tdm_prop", "tdm_prop1", "tdm_prop2", "tdm_der", "none"], 
                            default="none", help="Crosstalk model to use")
        parser.add_argument("--xifusim_xml_version", type=str, required=True, help="Version of the XIFU simulator XML files",
                            choices=["v5_20250621", "v3_20240215"])
        parser.add_argument("--focus", type=str, choices=["defoc", "infoc"], required=True, help="Focus setting (defoc or infoc)")
        parser.add_argument("--filter", type=str, choices=["be_filter", "nofilt"], required=True, help="Filter setting (be_filter or nofilt)")
        parser.add_argument("--verbose", type=int, default=0, help="Verbosity level")
        parser.add_argument("--nbrightest_pixels", type=int, default=384, help="Number of brightest pixels to consider for defocussed Crab simulations")
        parser.add_argument("--addMXS", type=lambda x: (str(x).lower() == 'true'), default=True, help="Whether to add MXS contribution")
        parser.add_argument("--addJitter", type=lambda x: (str(x).lower() == 'true'), default=True, help="Whether to add pointing jitter")
        parser.add_argument("--decimation_factor", type=int, default=10, help="Decimation factor for SIXTE simulation")
        args = parser.parse_args()
        return vars(args)

In [ ]:
params = get_parameters()
srcmodel = params["srcmodel"]
xcmfile = params["xcmfile"]
flux210_keV = params["flux210_keV"]
exposure = params["exposure"]
nonxbgd = params["nonxbgd"]
XTalk = params["XTalk"]
xifusim_xml_version = params["xifusim_xml_version"]
focus = params["focus"]
filter = params["filter"]
aux.verbose = params["verbose"]
nbrightest_pixels = params["nbrightest_pixels"]
addMXS = params["addMXS"]
addJitter = params["addJitter"]
decimation_factor = params["decimation_factor"]
if addJitter is False and decimation_factor > 1:
    raise ValueError("If addJitter is False, decimation_factor must be 1.")
elif addJitter is True and decimation_factor == 1:
    raise ValueError("If addJitter is True, decimation_factor must be greater than 1.")

if srcmodel is not None and xcmfile is not None:
    # raise warning: source model (srcmodel) will be derived from xcmfile
    aux.vprint(f"Warning: The source model '{srcmodel}' parameters will be derived from xcmfile {xcmfile}.")
if xcmfile is not None and not os.path.exists(xcmfile):
    raise FileNotFoundError(f"xcmfile {xcmfile} does not exist.")

if xcmfile is None:
    aux.vprint(f"xcmfile not provided. Using srcmodel \"{srcmodel}\" to set default xcmfile \"{srcmodel}.xcm\"")
    xcmfile = f"{srcmodel}.xcm"  # default xcmfile based on srcmodel
    if flux210_keV is None:
        # raise error: flux210_keV must be provided if xcmfile is not provided
        raise ValueError("flux210_keV must be provided if xcmfile is not provided.")
else:
    aux.vprint(f"Using provided xcmfile: {xcmfile}")
    if flux210_keV is None:
        # derive flux from xcmfile using XSPEC
        aux.vprint(f"Warning: flux210_keV not provided, getting it from xcmfile: {xcmfile}")
    else:
        aux.vprint(f"Using provided flux210_keV: {flux210_keV} erg/cm2/s")
# pretty print params
aux.vprint("Simulation parameters:")
for key, value in params.items():
    aux.vprint(f"  {key}: {value}")

### Create XSPEC model file if needed

In [ ]:
def create_crab_xcm(xcmfile):
    """
    Create an XCM file for the Crab spectrum.
    Parameters:
    - xcmfile: str, path to save the XCM file
    """

    AllModels.clear()
    Xset.clear()
    # define XSPEC parameters
    Xset.method = "leven"
    Xset.abund = "wilm"
    Xset.cosmo = "70 0. 0.7"
    Xset.xsect = "vern"
    Xset.statistics = "cstat"
    Xset.systematic = 0.0
    # Create the model: absorbed powerlaw
    mcmod = Model("TBabs*powerlaw")
    mcmod.TBabs.nH = 0.4
    mcmod.powerlaw.PhoIndex = 2.1
    mcmod.powerlaw.norm = 9.5

    # Save the model to the specified .xcm file path
    Xset.save(xcmfile)
    aux.vprint(f"Model saved to {xcmfile}.")

In [ ]:
def create_casa_xcm(xcmfile):
    """
    Create an XCM file for the CASA spectrum.
    ==> No frozen components have been set in this model. <==
    Parameters:
    - xcmfile: str, path to save the XCM file
    - flux210_keV: float or None, desired flux in the 2-10 keV band (erg/cm2/s)
    """

    AllModels.clear()
    Xset.clear()
    # define XSPEC parameters
    Xset.method = "leven"
    Xset.abund = "aspl"
    Xset.cosmo = "70 0. 0.7"
    Xset.xsect = "vern"
    Xset.statistics = "cstat"
    Xset.systematic = 0.0
    Xset.delta = 0.01
    Xset.NEIVERS = 3.
    # Create the model: TBabs(powerlaw + vnei + vnei)
    Emin = 2.0
    Emax = 10.0
    mcmod = Model("TBabs*(powerlaw + vnei + vnei)")
    mcmod.TBabs.nH = 1.08659
    mcmod.powerlaw.PhoIndex = 3.27
    mcmod.powerlaw.norm = 1.88184
    mcmod.vnei.kT = 1.77376   
    mcmod.vnei.H = 1   
    mcmod.vnei.He = 1   
    mcmod.vnei.C = 1
    mcmod.vnei.N = 9730.63
    mcmod.vnei.O = 2246.42
    mcmod.vnei.Ne = 14.1650
    mcmod.vnei.Mg = 0.239815
    mcmod.vnei.Si = 3991.07
    mcmod.vnei.S  = 4168.48
    mcmod.vnei.Ar = 0.636431
    mcmod.vnei.Ca = 4317.77 
    mcmod.vnei.Fe = 0.0
    mcmod.vnei.Ni = 0.0
    mcmod.vnei.Tau = 7.59804E+10
    mcmod.vnei.Redshift = 0.0
    mcmod.vnei.norm = 1.00000E-03
    mcmod.vnei_4.kT = 3.00000
    mcmod.vnei_4.H = 1.00000
    mcmod.vnei_4.He = 1.00000
    mcmod.vnei_4.C  = 0.0
    mcmod.vnei_4.N  = 0.0
    mcmod.vnei_4.O  = 0.0
    mcmod.vnei_4.Ne = 0.0
    mcmod.vnei_4.Mg = 0.0
    mcmod.vnei_4.Si = 0.0
    mcmod.vnei_4.S  = 0.0
    mcmod.vnei_4.Ar = 0.0
    mcmod.vnei_4.Ca = 0.0
    mcmod.vnei_4.Fe = 32.4576
    mcmod.vnei_4.Ni = 32.4576
    mcmod.vnei_4.Tau = 6.09249E+10
    mcmod.vnei_4.Redshift = 0.0
    mcmod.vnei_4.norm = 1.61969E-02

    # Save the model to the specified .xcm file path
    Xset.save(xcmfile)

### Calculate Flux (if not provided)

For the case of 1Crab simulation in 1 sec, add 5ct/s/pix to account for MXS

In [ ]:
# if spectral model file does not exist, create it
if xcmfile is None:
    if srcmodel == "crab":   
        create_crab_xcm(xcmfile)
    elif srcmodel == "casa":
        create_casa_xcm(xcmfile)

sys.stdout.flush() 
sys.stderr.flush()
print(f"Loading model file {xcmfile}")
# calculate flux xcmfile
AllModels.clear()
Xset.restore(xcmfile)
Emin = 2.0
Emax = 10.0
mcmod = AllModels(1)
AllModels.calcFlux(f"{Emin} {Emax}")
model_flux = AllModels(1).flux[0] # erg/cm2/s    
sys.stdout.flush() 
sys.stderr.flush()
print(f"Model loaded!\n")
    
if flux210_keV is None:
    flux210_keV = model_flux
    aux.vprint(f"Derived flux210_keV from xcmfile: {flux210_keV} erg/cm2/s")
else:
    aux.vprint(f"Using provided flux210_keV {flux210_keV} erg/cm2/s.")



### Set XML files for simulations (sixtesim and xifusim)   

By default xifusim does not add JITTER: all pulses are generated in phase with the sampling.    
To add the jiiter effect, two new parameters must be set in the xifusim call (t_clock and decimate_factor).    
For a subsampling factor of 10 (10 subsamples/sample):   
```
decimate_factor=10
t_clock=TCLOCK/decimate_factor
```
TCLOCK is defined in the config*.fits listed in the config XML file.

In [ ]:
## Set XML file for sixte simulation
#xmldir = f"{SIXTE}/share/sixte/instruments/athena-xifu/internal_design_goal"
xmldir = f"{SIXTE}/share/sixte/instruments/new-athena-xifu/internal_design_goal"
xml_sixtesim = f"{xmldir}/xifu_{filter}_{focus}.xml"
aux.vprint(f"Using sixtesim XML file: {xml_sixtesim}")

# Find name of (unique) xml file in indir directory
xml_xifusim = glob.glob(f"./config*{xifusim_xml_version}.xml")
if len(xml_xifusim) != 1:
    raise FileNotFoundError(f"Error: expected 1 XML file but found {len(xml_xifusim)}")
xml_xifusim = xml_xifusim[0]
aux.vprint(f"Using XIFUSIM XML file: {xml_xifusim}")

# read FITS config file from XML file xml_xifusim (detector tag, filename)
# get detector filename from xml_xifusim
import xml.etree.ElementTree as ET
tree = ET.parse(xml_xifusim)
root = tree.getroot()
config_fits_filename = None
for detector in root.findall('detector'):
    config_fits_filename = detector.get('filename')
    break  # assuming only one detector tag
if config_fits_filename is None:
    raise ValueError(f"Error: no detector filename found in XML file {xml_xifusim}")
aux.vprint(f"Using detector configuration file: {config_fits_filename}")
if addJitter:
    # insert jitter
    # read TCLOCK keyword from GEOCHANNELPARAM extension
    with fits.open(config_fits_filename, mode='update') as hdul:
        tclock_value = hdul['GEOCHANNELPARAM'].header['TCLOCK']
        aux.vprint(f"Original TCLOCK value: {tclock_value} s")
    new_tclock_value = tclock_value/decimation_factor  


In [ ]:
RA=0.
Dec=0.
sampling_rate=130210 #Hz
prebuff_xifusim=1500  #prebuffer samples for xifusim
fluxCrab210_keV = 2.0533E-8 # erg/cm2/s

brightest_pixels_file = None
filestring = ""
if srcmodel == "crab" and focus == "defoc": # case of 1Crab defocussed simulation (Crab_DB)
    simdir = "Crab_DB"
    brightest_pixels_file = f"./{simdir}/brightest_pix_ctrate.txt"
    flux_in_mcrab = flux210_keV / (fluxCrab210_keV / 1000)
    filestring = f"./{simdir}/{xifusim_xml_version}/crab_{filter}_{focus}_flux{flux_in_mcrab:.2f}mcrab"
    aux.vprint(f"Simulating {flux_in_mcrab:.1f} mCrab source for defocussed configuration")
elif srcmodel == "crab" and focus == "infoc": # case of 1Crab in focussed simulation (Point_Source_DB)
    simdir = "Point_Source_DB"
    flux_in_mcrab = flux210_keV / (fluxCrab210_keV / 1000)
    filestring = f"./{simdir}/{xifusim_xml_version}/crab_{filter}_{focus}_flux{flux_in_mcrab:.2f}mcrab"
elif srcmodel == "casa":
    simdir = "CasA_DB"
    filestring = f"./{simdir}/{xifusim_xml_version}/casa_{filter}_{focus}_flux{flux210_keV:.3e}cgs"

print(filestring)

## Do SIXTESIM simulation    
Files required:   
- XSPEC file 
- SIMPUT file   

### Create simput file

In [ ]:
# run simputfile to create the simput file
simputfile = f"{filestring}_simput.fits"
if not os.path.exists(simputfile):
        comm = (f'simputfile Simput={simputfile} RA={RA} Dec={Dec} '
                f'srcFlux={flux210_keV} Emin={Emin} Emax={Emax} '
                f'XSPECFile={xcmfile} clobber=yes')
        aux.vprint(f"Running {comm}")
        # Run the command through the subprocess module
        output_simputfile = run(comm, shell=True, capture_output=True)
        assert output_simputfile.returncode == 0, f"simputfile failed to run:{comm}; \nReason: {output_simputfile.stderr.decode()}"

        assert os.path.exists(simputfile), f"simputfile did not produce an output file"
else:
    aux.vprint(f"Simput file {simputfile} already exists. Skipping simputfile step.")

### Run SIXTESIM and get PIXIMPACT file 

In [ ]:
evtfile = f"{filestring}_evt.fits"
photfile = f"{filestring}_photon.fits"
impfile = f"{filestring}_impact.fits"
if not os.path.exists(evtfile) or not os.path.exists(photfile) or not os.path.exists(impfile):    
        comm = (f'sixtesim PhotonList={photfile} Simput={simputfile} '
                f'ImpactList={impfile} EvtFile={evtfile} doCrossTalk={XTalk} '
                f'XMLFile={xml_sixtesim} Background={nonxbgd} RA={RA} Dec={Dec} ' 
                f'Exposure={exposure} clobber=yes')
        aux.vprint(comm)
        output_sixtesim = run(comm, shell=True, capture_output=True)
        assert output_sixtesim.returncode == 0, f"sixtesim failed to run"
        assert os.path.exists(evtfile), f"sixtesim did not produce an output file"
        assert os.path.exists(photfile), f"sixtesim did not produce an output file"
        assert os.path.exists(impfile), f"sixtesim did not produce an output file"
else:
    aux.vprint(f"Sixtesim output files already exist. Skipping sixtesim step.")

### Get list of pixels with counts produced by sixtesim 

In [ ]:
#read column PIXID from evtfile and save to a list of unique pixels
hdulist = fits.open(evtfile, mode='update')
evtdata = hdulist[1].data.copy()
pixels_with_impacts = np.unique(evtdata["PIXID"])
print("SIXTE simulation results:")
aux.vprint(f"Number of pixels with impacts: {len(pixels_with_impacts)}")
hdulist.close()

# get pixels used and the events in each pixel
nimpacts_inpix = dict()
phid_impacts_inpix = dict()
for pixel in pixels_with_impacts:
    phid_impacts_inpix[pixel] = evtdata['PH_ID'][evtdata['PIXID'] == pixel]
    nimpacts_inpix[pixel] = len(phid_impacts_inpix[pixel])

#print number of impacts per pixel sorted by number of impacts
sim_brightest_pixels = sorted(nimpacts_inpix, key=nimpacts_inpix.get, reverse=True)
for pixel in sim_brightest_pixels:
    aux.vprint(f"Pixel {pixel}: {nimpacts_inpix[pixel]} impacts")


#print the PH_ID of impacts in pixels
for key, value in phid_impacts_inpix.items():
    aux.vprint(f"Pixel {key}: ")
    aux.vprint(f"      PH_ID:{value}")



### Add contribution of MXS source if needed

In [ ]:
if addMXS:
     # get the mean count rate per pixel in the nbrightest pixels from SIXTE simulation
     mean_sim_cts = np.mean([nimpacts_inpix.get(sim_brightest_pixels[i], 0) / exposure for i in range(nbrightest_pixels)])
     aux.vprint(f"Mean count rate in the {nbrightest_pixels} brightest pixels from SIXTE simulation: SIXTE={mean_sim_cts:.2f} cts/s/pixel")
     # calculate scaling factor to include MXS contribution of 5 ct/s/pix
     mean_sim_cts_with_mxs = mean_sim_cts + 5
     scaling_factor = mean_sim_cts_with_mxs/mean_sim_cts
     aux.vprint(f"Mean count rate including MXS contribution: SIXTE={mean_sim_cts_with_mxs:.2f} cts/s/pixel")
     aux.vprint(f"Scaling factor to include MXS contribution: {scaling_factor:.2f}")

     ## REDO SIXTE SIMULATION WITH ADJUSTED FLUX TO INCLUDE MXS CONTRIBUTION
     # adjust flux 210 keV to include MXS contribution
     desired_flux_with_mxs = flux210_keV * scaling_factor
     flux210_keV_mxs = desired_flux_with_mxs
     aux.vprint(f"Simulating source with flux210_keV={flux210_keV_mxs:.3e} erg/cm2/s to include MXS contribution")
   

In [ ]:
if addMXS:  
     # run simputfile to create the simput file
     # ====================================================
     filestring = f"{filestring}_withMXS"
     simputfile = f"{filestring}_simput.fits"
     if not os.path.exists(simputfile):
          comm = (f'simputfile Simput={simputfile} RA={RA} Dec={Dec} '
                         f'srcFlux={flux210_keV_mxs} Emin={Emin} Emax={Emax} '
                         f'XSPECFile={xcmfile} clobber=yes')
          aux.vprint(f"Running {comm}")
          # Run the command through the subprocess module
          output_simputfile = run(comm, shell=True, capture_output=True)
          assert output_simputfile.returncode == 0, f"simputfile failed to run:{comm}; \nReason: {output_simputfile.stderr.decode()}"

          assert os.path.exists(simputfile), f"simputfile did not produce an output file"
     else:
          aux.vprint(f"Simput file {simputfile} already exists. Skipping simputfile step.")



In [ ]:
if addMXS:
     # ====================================================================
     # run sixtesim with the new simput file (to include MXS contribution)
     # ====================================================================
     evtfile = f"{filestring}_evt.fits"
     photfile = f"{filestring}_photon.fits"
     impfile = f"{filestring}_impact.fits"
     if not os.path.exists(evtfile) or not os.path.exists(photfile) or not os.path.exists(impfile):    
          comm = (f'sixtesim PhotonList={photfile} Simput={simputfile} '
                    f'ImpactList={impfile} EvtFile={evtfile} doCrossTalk={XTalk} '
                    f'XMLFile={xml_sixtesim} Background={nonxbgd} RA={RA} Dec={Dec} ' 
                    f'Exposure={exposure} clobber=yes')
          aux.vprint(comm)
          output_sixtesim = run(comm, shell=True, capture_output=True)
          assert output_sixtesim.returncode == 0, f"sixtesim failed to run"
          assert os.path.exists(evtfile), f"sixtesim did not produce an output file"
          assert os.path.exists(photfile), f"sixtesim did not produce an output file"
          assert os.path.exists(impfile), f"sixtesim did not produce an output file"
     else:
          aux.vprint(f"Sixtesim output files already exist. Skipping sixtesim step.")


In [ ]:
if addMXS:
     # compare number of impacts in each pixel before and after adding MXS contribution
     hdulist = fits.open(evtfile, mode='update')
     evtdata = hdulist[1].data.copy()
     pixels_with_impacts_mxs = np.unique(evtdata["PIXID"])
     print(f"SIXTE simulation results (including MXS contribution, scaling factor={scaling_factor:.2f}):")
     aux.vprint(f"Number of pixels with impacts: {len(pixels_with_impacts_mxs)}")
     hdulist.close()

     # get pixels used and the events in each pixel
     nimpacts_inpix_mxs = dict()
     phid_impacts_inpix_mxs = dict()
     for pixel in pixels_with_impacts_mxs:
          phid_impacts_inpix_mxs[pixel] = evtdata['PH_ID'][evtdata['PIXID'] == pixel]
          nimpacts_inpix_mxs[pixel] = len(phid_impacts_inpix_mxs[pixel])
     #print number of impacts per pixel sorted by number of impacts pre and post MXS addition in a table

     for pixel in sim_brightest_pixels:
          aux.vprint(f"Pixel {pixel}: SIXTE={nimpacts_inpix.get(pixel, 0)} impacts; SIXTE+MXS={nimpacts_inpix_mxs.get(pixel, 0)} impacts")
          


### Check distribution of impacts w/ EC's input file

In [ ]:
if brightest_pixels_file is not None and aux.is_notebook():
    print(f"Comparing SIXTE simulation results with EC requirements:")
    # read ascii file with brightest pixels countrate into an astropy table: 1 column with countrate (no header)
    brightest_pixels_cts = Table.read(brightest_pixels_file, format='ascii.no_header')
    # set column name to IN_CTRATE
    brightest_pixels_cts.rename_column('col1', 'IN_CTRATE')
    # add 5 cts/s to each pixel
    brightest_pixels_cts['IN_CTRATE'] += 5
    # sort column in descending order
    brightest_pixels_cts.sort('IN_CTRATE', reverse=True)

    # print the counts in the nbrightest pixels from SIXTE simulation and from EC requirements in a table of two columns
    for i in range(nbrightest_pixels):
        sim_cts = nimpacts_inpix.get(sim_brightest_pixels[i], 0)
        ec_cts = brightest_pixels_cts['IN_CTRATE'][i]
        aux.vprint(f"Pixel rank {i+1}: SIXTE={sim_cts:.2f} cts, EC_cts={ec_cts:.2f} cts")
    


### Read sixtesim output data

In [ ]:
# open sixtesim ImpactList and read data 
hdulist = fits.open(impfile)
impdata = hdulist[1].data.copy()
hdulist.close()
#vprint(impdata)

## Do XIFUSIM simulation  for the simulated brightest pixels

In [ ]:
aux.vprint(f"Simulated Brightest pixels: {sim_brightest_pixels}")

### Extract a piximpact file for each interesting pixel   
- create a subsample of the piximpact file selecting only those rows where PH_ID is in the list of the impacts in the pixel   
- copy src impacts from impact file and bkgs from event file (background are not in the impact list, only in the event file)

In [ ]:
for ipix in range(nbrightest_pixels):
    ipixel = sim_brightest_pixels[ipix]
    aux.vprint(f"Checking existence of piximpact file for pixel {ipixel}")
    # create a subsample of the piximpact file selecting only those rows where PH_ID is in 
    # the list of the impacts in the pixel
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    # if file does not exist, create it
    if not os.path.exists(piximpactfile):
        # create a mask to select only the rows with the impacts in this pixel
        mask = np.isin(impdata['PH_ID'], phid_impacts_inpix[ipixel])
        # create a new table with the selected rows
        newtable = Table(impdata[mask])
        # sort newtable according to TIME
        newtable.sort('TIME')
            
        # add new columns X,Y,U,V, GRADE1, GRADE2, TOTALEN with the value 0 and PIXID with the value of ipixel
        newtable['X'] = 0.
        newtable['Y'] = 0.
        newtable['U'] = 0.
        newtable['V'] = 0.
        newtable['GRADE1'] = 0
        newtable['GRADE2'] = 0
        newtable['TOTALEN'] = 0
        newtable['PIXID'] = 1 #requirement for xifusim

        # name the new table 'PIXELIMPACT'
        newtable.meta['EXTNAME'] = 'PIXELIMPACT'
    
        # write the new table to a new FITS file
        newtable.write(piximpactfile, format='fits', overwrite=True)

        # print the name of the new file rewriting the output line
        aux.vprint(f"Created {piximpactfile} for pixel {ipixel}")


### Run the xifusim simulation (with XML for single pixel)   
    - simulate time between min and max TIME in piximpact   
    - xifusim simulation    
    - re-establish correct PIXID    

In [ ]:
#for each piximpact file, run xifusim
prebuffer = 1500
phsims_inpix = dict()
nphsims_inpix = dict()
skipped_photons_inpix = dict()
skipped_xifusim = []   

for ipix in range(nbrightest_pixels):
    ipixel = sim_brightest_pixels[ipix]
    piximpactfile = f"{filestring}_pixel{ipixel}_piximpact.fits"
    # read the piximpact file and get TIME values
    with fits.open(piximpactfile) as hdulist_piximpact:
        piximpactdata = hdulist_piximpact[1].data.copy()
    xifusimfile = f"{filestring}_pixel{ipixel}_xifusim.fits"
    if not os.path.exists(xifusimfile):
        #calculate minimum and maximum time for impacts in the pixel
        mintime = np.min(piximpactdata['TIME'])
        maxtime = np.max(piximpactdata['TIME'])
        expos_init = 1.e-6
        expos_fin = exposure
    
        #create xifusim name based on input parameters  
        
        comm = (f'xifusim PixImpList={piximpactfile} Streamfile={xifusimfile} '
                f'tstart={expos_init} tstop={expos_fin} '
                f'writeDRE=1 writeTrigger=0 decimate_factor={decimation_factor} '
                f't_clock={new_tclock_value:.6f} '
                f'XMLfilename={xml_xifusim} clobber=yes ')
        
        aux.vprint(f"  Doing simulation for pixel {ipixel} with {len(piximpactdata)} impacts ({nimpacts_inpix[ipixel]} TOTAL impacts)")
        print(f"Running {comm}")
        output_xifusim = run(comm, shell=True, capture_output=True)
        #assert output_xifusim.returncode == 0, f"xifusim failed to run: {comm}"
        assert output_xifusim.returncode == 0, print(f"xifusim failed to run: {comm} => ERROR COED: {output_xifusim.stdout.decode()}")
        assert os.path.exists(xifusimfile), f"xifusim did not produce an output file"

        # re-write correct PIXID in the xifusim file
        with fits.open(xifusimfile, mode='update') as hdulist:
            xifusimdata = hdulist[1].data
            xifusimdata['PIXID'] = ipixel
            hdulist.flush()